<a href="https://colab.research.google.com/github/lukaszplust/Projects/blob/main/proba_1_sbd.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# BUFFER_SIZE - określa liczbę rekordów, które będą przechowywane w buforze w jednym momencie.
# Buforowanie poprawia wydajność operacji wejścia-wyjścia, minimalizując liczbę operacji odczytu i zapisu na dysk poprzez grupowanie ich w większe bloki.
BUFFER_SIZE = 32


# SET_BYTES_SIZE - określa rozmiar w bajtach pojedynczego rekordu bez dodatkowych znaków. Zakłada się, że rekord zawiera dane o stałej długości
SET_BYTES_SIZE = 15

# Ta stała określa rozmiar w bajtach pojedynczego rekordu, włączając dodatkowy znak (prawdopodobnie nowa linia lub znak końca rekordu).
# Zazwyczaj jest to znak '\n' (nowa linia), który oddziela rekordy w pliku tekstowym
RECORD_BYTES_SIZE = SET_BYTES_SIZE + 1

# Ta stała określa całkowity rozmiar bufora w bajtach.
# Jest to iloczyn liczby rekordów w buforze (BUFFER_SIZE) i rozmiaru jednego rekordu w bajtach (RECORD_BYTES_SIZE).
# Bufor o takim rozmiarze będzie używany do operacji odczytu i zapisu blokowego, co zwiększa efektywność przez minimalizację liczby operacji I/O
BYTES_BUFFER_SIZE = BUFFER_SIZE * RECORD_BYTES_SIZE




Należy zaimplementować symulację odczytu oraz zapisu blokowego jako oddzielną
warstwę logiki programu. Warstwa ta powinna udostępniać warstwie sorowania co najmniej 2
operacje: odczytu oraz zapisu pojedynczego rekordu.

In [3]:
class ReadBuffer:

  def __init__(self, file_path):

    # pozycja odczytu w buforze
    self.read_pos = 0
    # rozmiar bufora, ustawiony na BUFFER_SIZE
    self.size = BUFFER_SIZE
    # liczba rekordów obecnie załadowanych do bufora
    self.loaded_size = 0
    # lista przechowująca rekordy załadowane do bufora
    self.buffer = []
    # ścieżka do pliku, z którego będą odczytywane rekordy
    self.file_path = file_path
    # pozycja odczytu w pliku
    self.file_pos = 0
    # całkowity rozmiar pliku (w bajtach)
    self.file_size = os.path.getsize(file_path)
    # licznik operacji odczytu z dysku
    self.disk_reads_count = 0
    # inicjalnie ładuje pierwszą porcję danych do bufora
    self.load_next()


  # ODCZYT REKORDÓW: Metoda read_next zwraca następny rekord
  def read_next(self):

    # sprawdzam, czy są jeszcze rekordy do odczytu
    if not self.has_more():
        return None

    # jeśli są jakieś rekordy do odczytu to zwracam następny rekord z bufora
    res_record = self.buffer[self.read_pos]
    self.read_pos += 1

    # jeśli pozycja odczytu osiągnie rozmiar bufora
    if self.read_pos == self.size:

        # ładuje kolejną porcję danych do bufora (self.load_next())
        self.load_next()

        # resetuje self.read_pos
        self.read_pos = 0


    return res_record


  # Muszę sprawdzić, czy są jeszcze dane do odczytu w pliku lub w buforze.
  def has_more(self):
        return (self.file_pos < self.file_size or self.read_pos < self.loaded_size)

  # PODGLĄD NASTĘPNEGO REKORDU: Metoda peek pozwala na podgląd następnego rekordu bez przesuwania wskaźnika odczytu
  # Zwracam następny rekord bez zmiany pozycji odczytu. Jeśli bufor jest pusty, zwracam None.
  def peek(self):

    # pozycja odczytu w pliku = liczba rekordów obecnie załadowanych do bufora
    if self.read_pos == self.loaded_size:
        return None
    result = self.buffer[self.read_pos]
    return result

  # ŁADOWANIE KOLEJNEJ PORCJI DANYCH: Metoda load_next ładuje nową porcję danych z pliku do bufora.
  def load_next(self):

        self.buffer = []
        # buffering=0 disables buffering, it is desired, because buffering is implemented here, in code

        # otwieram plik w trybie binarnym bez buforowania (buffering=0)
        file = open(self.file_path, "rb", buffering=0)

        # przesuwam wskaźnik odczytu pliku do self.file_pos
        file.seek(self.file_pos)

        # odczytuje maksymalnie BYTES_BUFFER_SIZE bajtów lub pozostałe bajty do końca pliku (self.file_size - self.file_pos)
        bytes_to_read = min(BYTES_BUFFER_SIZE,self.file_size - self.file_pos)

        temp_buffer = file.read(bytes_to_read)

        # sprawdzam, czy liczba odczytanych bajtów jest wielokrotnością rozmiaru rekordu (RECORD_BYTES_SIZE)
        if len(temp_buffer) % RECORD_BYTES_SIZE != 0:
            raise Exception("Read bytes are not multiply of record size")

        # aktualizuje pozycję odczytu pliku (self.file_pos)
        self.file_pos += bytes_to_read

        # obliczam liczbę rekordów załadowanych do bufora (self.loaded_size)
        self.loaded_size = bytes_to_read / RECORD_BYTES_SIZE

        # Konwertuje odczytane bajty na listę rekordów
        temp_ints = list(temp_buffer)

        # dodaje liste rekordów do bufora (self.buffer)
        for i in range(len(temp_buffer) // RECORD_BYTES_SIZE):
            record_ints = temp_ints[
                          RECORD_BYTES_SIZE * i:RECORD_BYTES_SIZE * (i + 1)
                          ]
            self.buffer.append(Record.load_from_ints(record_ints))

        # zamykam plik i zwiększam licznik operacji odczytu z dysku (self.disk_reads_count)
        file.close()
        self.disk_reads_count += 1



        # ITERACJA: Klasa jest iterowalna dzięki metodom __iter__ i __next__
        # Metody __iter__ i __next__ są częścią protokołu iteracji w Pythonie. Dzięki tym metodom można sprawić, że obiekt będzie iterowalny,
        # co oznacza, że można go używać w konstrukcjach takich jak pętle for, list comprehensions,
        # funkcje map, filter oraz inne operacje, które wymagają iterowalnych obiektów

        # INICJALIZACJA BUFORA: Konstruktor __init__ otwiera plik i ładuje pierwszą porcję danych do bufora
        # __iter__: Zwraca iterator (sam obiekt ReadBuffer)
        def __iter__(self):
          return self

        # __next__: Zwraca następny rekord z bufora. Jeśli nie ma więcej rekordów, podnosi wyjątek StopIteration
        def __next__(self):
          next_record = self.read_next()
          if next_record is None:
            raise StopIteration
        return next_record
